In [2]:
!pip install --quiet arcgis

In [3]:
import os
from arcgis.gis import GIS;
import requests;
import arcpy;
import shutil;
import urllib;
import zipfile;

In [6]:
ned_lst = r"https://viewer.nationalmap.gov/tnmaccess/api/products";

response = requests.get(
     ned_lst
    ,params={
         'bbox':'-180,-17,-35,72'
        ,'datasets': 'National Elevation Dataset (NED) 1/3 arc-second'
        ,'prodFormats': 'IMG'
        ,'prodExtents': '1 x 1 degree'
    }
);
resp_json = response.json();

ary = [];
for item in resp_json['items']:
    
    if item['downloadURL'][:26] == 'ftp://rockyftp.cr.usgs.gov':
        continue;
    
    ary.append(item);

str(resp_json['total']) + ' ' + str(len(ary))


'1376 1363'

In [7]:
src = r"F:\USGS\NED13\src";
if not os.path.exists(src):
    os.makedirs(src);
    
trg = r"F:\USGS\NED13\trg";
if not os.path.exists(trg):
    os.makedirs(trg);
    

In [8]:
cnt = 0;
for item in ary:
    fn = item['downloadURL'].split('/')[-1];
    
    if not os.path.exists(src + os.sep + fn):
        
        #print("Downloading " + fn);
        urllib.request.urlretrieve(
             item['downloadURL']
            ,src + os.sep + fn
        );
        cnt = cnt + 1;
        
str(cnt) + " files downloaded"

'0 files downloaded'

In [10]:
cnt = 0;
for item in ary:
    fn = item['downloadURL'].split('/')[-1];
    dr = os.path.splitext(fn)[0];
    
    if  os.path.exists(src + os.sep + fn)        \
    and not os.path.exists(trg + os.sep + dr):
        
        os.mkdir(trg + os.sep + dr);
        
        print("Unzipping " + fn);
        with zipfile.ZipFile(src + os.sep + fn,'r') as zip_ref:
            zip_ref.extractall(trg + os.sep + dr);
        
        if os.path.exists(trg + os.sep + dr + os.sep + dr):
            os.rename(
                 trg + os.sep + dr
                ,trg + os.sep + dr + 'X'
            );
            
            shutil.move(
                 trg + os.sep + dr + 'X' + os.sep + dr
                ,trg + os.sep + dr
            );
            
            shutil.rmtree(trg + os.sep + dr + 'X');
        
        cnt = cnt + 1;
        
str(cnt) + " files unzipped"  

'0 files unzipped'

In [12]:
cnt = 0;
for item in ary:
    fn = item['downloadURL'].split('/')[-1];
    dr = os.path.splitext(fn)[0];
    
    if len(dr) == 7:
        img = 'img' + dr + '_13.img';
    else:
        img = dr + '.img';
    
    props = arcpy.GetRasterProperties_management(
         in_raster     = trg + os.sep + dr + os.sep + img
        ,property_type = 'VALUETYPE'
    );
    
    vt = props.getOutput(0);
    if vt != '9':
        raise Exception(img + ' ' + vt);

'files are good'

'files are good'